# Real World Problems Series

### Customer Support AI Agent - Urgent Booking Changes

Automate cancellations/rescheduling using mock travel agency APIs and LangGraph.

**Date**: 30 April, 2025

### 1. Objective
Build an AI agent to handle urgent booking changes (cancellations/rescheduling) by:
- Integrating with travel agency APIs.
- Using LangGraph for workflow orchestration.
- Ensuring secure authentication and user confirmation.

### 2. Setup
Install Dependencies

In [ ]:
!pip install langgraph requests fastapi uvicorn python-dotenv nest-asyncio pyngrok > /dev/null

In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")
_set_env("NGROK_AUTH_TOKEN")

### Mock APIs (FastAPI)
Create a mock server to simulate the travel agency's backend:

In [ ]:
from fastapi import FastAPI, HTTPException, Header
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok

app = FastAPI()

# Authenticate ngrok
ngrok.set_auth_token(os.getenv("NGROK_AUTH_TOKEN"))

# Mock database
mock_bookings = {
    "BOOKING123": {
        "user_id": "USER456",
        "status": "confirmed",
        "flight": "NYC-LON 2024-10-20"
    }
}

# --- Mock APIs ---
@app.get("/bookings/{booking_id}")
def get_booking(booking_id: str, api_key: str = Header(...)):
  if booking_id not in mock_bookings:
    raise HTTPException(status_code=404, detail="Booking not found")
  return mock_bookings[booking_id]

@app.post("/bookings/{booking_id}/cancel")
def cancel_booking(booking_id: str, api_key: str = Header(...)):
  if booking_id not in mock_bookings:
    raise HTTPException(status_code=404, detail="Booking not found")
  mock_bookings[booking_id]["status"] = "cancelled"
  return {"message": "Cancellation successful"}

# Start server via ngrok (for Colab Compatibility)
ngrok_tunnel = ngrok.connect(8000)
print(f"API URL: {ngrok_tunnel.public_url}")

import nest_asyncio
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

### 3. Core Components

**State Schema**

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
import requests

class AgentState(TypedDict):
    user_input: str
    booking_id: Optional[str]
    api_key: Optional[str]
    intent: Optional[str]
    booking_details: Optional[dict]
    confirmation: Optional[bool]
    error: Optional[str]

**Nodes**

In [ ]:
# ---- Node 1: Parse User Input ----
def parse_input(state: AgentState) -> AgentState:
  """Extract booking ID/intent from natural language."""
  text = state["user_input"].lower()
  state["intent"] = "cancel_booking" if "cancel" in text else "unknown"
  state["booking_id"] = "BOOKING123" # Mock extraction (use LLM in production)
  return state

# ---- Node 2: Authenticate ----
def authenticate(state: AgentState) -> AgentState:
  """Validate API key against mock system."""
  if state.get("api_key") != "SECRET_KEY_123":
    state["error"] = "Invalid API key"
  return state

# ---- Node 3: Fetch Booking Details ----
def fetch_booking(state: AgentState) -> AgentState:
  """Call mock API to retrieve booking data."""
  if state.get("error"):
    return state # Skip if auth failed
  
  booking_id = state["booking_id"]
  response = requests.get(
      f"{ngrok_tunnel.public_url}/bookings/{booking_id}",
      headers={"api_key": state["api_key"]}
  )
  if response.status_code == 200:
    state["booking_details"] = response.json()
  else:
    state["error"] = response.json()["detail"]
  return state

# ---- Node 4: Confirm Action ----
def confirm_action(state: AgentState) -> AgentState:
  """Seek user confirmation (mock UI interaction)."""
  if not state.get("error"):
    print(f"PROMPT: Cancel booking {state['booking_id']}? [yes/no]")
    state["confirmation"] = True # Simulate user input "yes"
  return state

# ---- Node 5: Process Cancellation ----
def process_cancellation(state: AgentState) -> AgentState:
  """Execute cancellation via mock API."""
  if state.get("confirmation") and not state.get("error"):
    response = requests.post(
        f"{ngrok_tunnel.public_url}/bookings/{state['booking_id']}/cancel",
        headers={"api_key": state["api_key"]}
    )
    if response.status_code != 200:
      state["error"] = "Cancellation failed"
  return state

# ---- Node 6: Error Handler ----
def handle_error(state: AgentState) -> AgentState:
  """Route errors to escalation or retry."""
  if state.get("error"):
    print(f"ERROR: {state['error']} - Escalating to human agent.")
  return state

### 4. Workflow Graph

In [ ]:
# Build LangGraph
builder = StateGraph(AgentState)

# Add Nodes
builder.add_node("parse_input", parse_input)
builder.add_node("authenticate", authenticate)
builder.add_node("fetch_booking", fetch_booking)
builder.add_node("confirm_action", confirm_action)
builder.add_node("process_cancellation", process_cancellation)
builder.add_node("error_handler", handle_error)

# Define edges
builder.set_entry_point("parse_input")
builder.add_edge("parse_input", "authenticate")
builder.add_edge("authenticate", "fetch_booking")
builder.add_edge("fetch_booking", "confirm_action")
builder.add_edge("confirm_action", "process_cancellation")
builder.add_edge("process_cancellation", "error_handler")
builder.add_edge("error_handler", END)

# Conditional edges (example)
# builder.add_conditional_edges("authenticate", lambda x: "error_handler" if x.get("error") else "fetch_booking")

agent = builder.compile()

### 5. Testing & Validation

**Test Case 1: Successful Cancellation**

In [ ]:
initial_state = {
    "user_input": "Urgent! Cancel my booking BOOKING123",
    "api_key": "SECRET_KEY_123"
}

result = agent.invoke(initial_state)
print("Final State:", result)
print("Booking Status:", mock_bookings["BOOKING123"]["status"]) # Should be "cancelled"

**Test Case 2: Authentication Failure**

In [ ]:
initial_state = {
    "user_input": "Cancel BOOKING123",
    "api_key": "WRONG_KEY"
}

result = agent.invoke(initial_state) # Should escalate error